In [ ]:
%cd /content
!rm -rf DeepLearningSystem_CMU10414
!git clone https://github.com/yyj6666667/DeepLearningSystem_CMU10414
%cd DeepLearningSystem_CMU10414/hw4


!pip3 install pybind11

!make clean
!make

%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd


In [2]:
import sys
sys.path.append('./python')
import urllib.request
import os

#!mkdir -p './data/ptb'
## Download Penn Treebank dataset
#ptb_data = "https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb."
#for f in ['train.txt', 'test.txt', 'valid.txt']:
#    if not os.path.exists(os.path.join('./data/ptb', f)):
#        urllib.request.urlretrieve(ptb_data + f, os.path.join('./data/ptb', f))

# Download CIFAR-10 dataset
if not os.path.isdir("./data/cifar-10-batches-py"):
    urllib.request.urlretrieve("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", "./data/cifar-10-python.tar.gz")
    !tar -xvzf './data/cifar-10-python.tar.gz' -C './data'

tar: Error opening archive: Failed to open ''./data/cifar-10-python.tar.gz''


以下用于快速验证

In [4]:
import numpy as np
sys.path.append("/python")
from needle import cpu, data, nn, optim, Tensor
np.random.seed(0)
device = cpu()
dataset = data.CIFAR10Dataset("./data/cifar-10-batches-py", train=True)
dataloader = data.DataLoader(
    dataset=dataset,
    batch_size=128,
    shuffle=True
)

from apps.models import ResNet9
np.random.seed(0)

def one_iter_of_cifar10_training(dataloader, model, niter=1, loss_fn=None, opt=None):
    if loss_fn is None:
        loss_fn = nn.SoftmaxLoss()
    if opt is None:
        opt = optim.Adam(model.parameters())
    
    np.random.seed(4)
    model.train()
    total_correct, total_loss, total_samples = 0, 0, 0
    i = 0
    for batch in dataloader:
        if i >= niter: 
            break
        opt.reset_grad()
        X, y = batch
        X, y = Tensor(X, device=device), Tensor(y, device=device)
        print("X, y is loaded into cpu")
        out = model(X)
        print(f"X range: [{X.numpy().min()}, {X.numpy().max()}]")
        print(f"Model output range: [{out.numpy().min()}, {out.numpy().max()}]")

        cur_correct = np.sum(np.argmax(out.numpy(), axis=1) == y.numpy())
        total_correct += cur_correct
        total_samples += y.shape[0]

        loss = loss_fn(out, y)
        total_loss += loss.data.numpy() * y.shape[0]

        loss.backward()
        opt.step()
        
        i += 1
        batch_cor = cur_correct / (y.shape[0])
        print(f"for batch{i}: batch_correct is {batch_cor}, batch_loss is {loss}")
    return total_correct/(total_samples), total_loss/total_samples

model = ResNet9(device=device, dtype="float32")
out = one_iter_of_cifar10_training(
    dataloader, 
    model, 
    niter = 10,
    opt=optim.Adam(model.parameters(), lr=0.000005, weight_decay=0.01)
)
print(f"one epoch: correct is {out[0]}, total_loss is {out[1]}")

X, y is loaded into cpu
X range: [0.0, 1.0]
Model output range: [-4.662904262542725, 4.381856918334961]
for batch1: batch_correct is 0.0546875, batch_loss is [3.1863413]
X, y is loaded into cpu
X range: [0.0, 1.0]
Model output range: [-4.972354412078857, 5.962058067321777]
for batch2: batch_correct is 0.109375, batch_loss is [3.0185716]
X, y is loaded into cpu
X range: [0.0, 1.0]
Model output range: [-5.948658466339111, 4.246920585632324]
for batch3: batch_correct is 0.09375, batch_loss is [3.0377967]
X, y is loaded into cpu
X range: [0.0, 1.0]
Model output range: [-7.121026039123535, 4.551535129547119]
for batch4: batch_correct is 0.1484375, batch_loss is [2.9306831]
X, y is loaded into cpu
X range: [0.0, 1.0]
Model output range: [-6.414260387420654, 4.596076488494873]
for batch5: batch_correct is 0.1328125, batch_loss is [3.0671468]
X, y is loaded into cpu
X range: [0.0, 1.0]
Model output range: [-5.001521110534668, 4.145654201507568]
for batch6: batch_correct is 0.1015625, batch_los